In [50]:
from hashlib import sha256

q = 0xFFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFF 
a = 0xFFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFC 
b = 0x28E9FA9E9D9F5E344D5A9E4BCF6509A7F39789F515AB8F92DDBCBD414D940E93
Gx= 0x32C4AE2C1F1981195F9904466A39C9948FE30BBFF2660BE1715A4589334C74C7 
Gy= 0xBC3736A2F4F6779C59BDCEE36B692153D0A9877CC62A474002DF32E52139F0A0

E = EllipticCurve(Zmod(q), [a, b])
G = E(Gx, Gy)
ord_ = E.order()


def Agg(sig):
    R = E(0)
    s = 0
    for item in sig:
        R += item[0]
        s += item[1]
    return (R, s%ord_)


def KAgg(pk):
    strx = ''
    for X in pk:
        strx += str(X)
    L = sha256(strx.encode()).digest()
    res = E(0)
    for X in pk:
        res += int(sha256(L+str(X).encode()).hexdigest(), 16)*X
    return res


def multi_schnorr_sign(R_l, X_l, sk, r, m, idx):
    R = E(0)
    for R_ in R_l:
        R += R_
    strx = ''
    for X_ in X_l:
        strx += str(X_)
    L = sha256(strx.encode()).digest()
    X = E(0)
    for X_ in X_l:
        X += int(sha256(L+str(X_).encode()).hexdigest(), 16)*X_

    s = r+int(sha256(str(X).encode()+str(R).encode()+m.encode()).hexdigest(), 16)\
    *int(sha256(L+str(X_l[idx]).encode()).hexdigest(), 16)*sk
    return (R_l[idx], s%ord_)

def multi_schnorr_verify(sig, pk, m):
    if sig[1]*G == sig[0]+int(sha256(str(pk).encode()+ \
                                 str(sig[0]).encode()+m.encode()).hexdigest(), 16)*pk:
        return True
    return False

def test():
    msg = 'hash_hash'
    n = 2
    all_x = [randint(1, ord_-1) for _ in range(n)]
    all_X = [x*G for x in all_x]
    all_r = [randint(1, ord_-1) for _ in range(n)]
    all_R = [r*G for r in all_r]
    sigma = []
    for _ in range(n):
        sigma.append(multi_schnorr_sign(all_R, all_X, all_x[_], \
                                  all_r[_], msg, _))
    sig = Agg(sigma)
    pk = KAgg(all_X)
    
    print(multi_schnorr_verify(sig, pk, msg))
    

if __name__ == '__main__':
    test()
        
    
    
    
    

True
